# Compliance Checks
To what degree do the models do or refuse their tasks?

In [1]:
STUDY_FOLDER = "introspecting_on_shifted_object_level_random_data" # 🔵 within exp/
CONDITIONS = { 
    # see `analysis/loading_data.py` for details
    # ("language_model","model"): ["gpt-4-0613"],
    # ("dataset","n_shot"): [0],
    # ("prompt,method"): ["object_level/minimal"],
}

In [2]:
from pathlib import Path
import subprocess
import sys

In [3]:
# Run the git command to get the repository root directory
REPO_DIR = subprocess.check_output(["git", "rev-parse", "--show-toplevel"]).decode().strip()

print("Repository directory:", REPO_DIR)
sys.path.append(REPO_DIR)

Repository directory: /home/felix/introspection_self_prediction_astra


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import words
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from evals.analysis.analysis_helpers import merge_object_and_meta_dfs, create_df_from_configs, fill_df_with_function, get_pretty_name, pretty_print_config
from evals.analysis.loading_data import load_dfs_with_filter

In [6]:
# Set the display option to None to show all content
pd.set_option('display.max_colwidth', 200)

In [7]:
# set color palette
palette = sns.color_palette("Set1")
sns.set_palette(palette)

In [8]:
# Set the directory for the data
EXPDIR = Path(REPO_DIR) / "exp"

In [9]:
# load the dataframes with configs as keys while keeping the non-compliant ones
dfs = load_dfs_with_filter(EXPDIR / STUDY_FOLDER, CONDITIONS, exclude_noncompliant=False)

In [10]:
def is_base_config(config):
    return "object" in config["prompt"]["method"]

In [11]:
object_dfs = {config: df for config, df in dfs.items() if is_base_config(config)}
meta_dfs = {config: df for config, df in dfs.items() if not is_base_config(config)}
print(f"Loaded {len(object_dfs)} base and {len(meta_dfs)} self-prediction dataframes")

Loaded 0 base and 0 self-prediction dataframes


In [13]:
# create results dataframe
results = create_df_from_configs(dfs.keys())
# results.sort_values(by="language_model_model", inplace=True)

How many responses are non-compliant?

In [14]:
def avg_compliance(df):
    return (df["compliance"] == True).mean()

In [15]:
# fill the results dataframe with the accuracy and t-statistic
fill_df_with_function(dfs, avg_compliance, "avg_compliance", results)

avg_compliance: 0it [00:00, ?it/s]


In [16]:
results.drop(columns=["config"]).sort_values(by="avg_compliance", ascending=False).style.hide(axis="index").background_gradient(subset=["avg_compliance"], cmap="RdYlGn", low=0.0, high=1.0)

KeyError: "['config'] not found in axis"

Making plots

In [ ]:
try:
    sns.pointplot(data=results, x="dataset_n_shot", y="avg_compliance", hue="language_model_model")
    plt.title("Average compliance per number of few-shot examples")
    plt.xlabel("Number of few-shot examples")
    plt.ylabel("Average compliance")
    plt.ylim(-0.025, 1.025)
    plt.show()
except ValueError as e:
    print(e)

In [ ]:
# a bar plot with bars for each language model and prompt method and also n_shot
sns.barplot(data=results, x="language_model_model", y="avg_compliance", hue="prompt_method")
plt.title("Average compliance per language model and prompt method")
plt.xlabel("Language model")
plt.ylabel("Average compliance")
plt.ylim(0,1)
plt.show()


### Qualitative Analysis

In [ ]:
# for each df, what are the most common reasons for non-compliance?
def get_most_common_reasons(df):
    df = df[df["compliance"] != True] 
    return df["compliance"].value_counts().head(10)

fill_df_with_function(dfs, get_most_common_reasons, "most_common_reasons", results)

In [ ]:
# show the most common reasons for non-compliance
for config, row in results.iterrows():
    pretty_print_config(config=config)
    print(row["most_common_reasons"])
    print()

In [ ]:
# for each df, sample 10 non-compliant examples
def sample_non_compliant(df):
    try:
        return df[df["compliance"] != True].sample(10)["response"]
    except:
        return None

fill_df_with_function(dfs, sample_non_compliant, "sample_non_compliant", results)

In [ ]:
# show the sample non-compliant examples
for config, row in results.iterrows():
    pretty_print_config(config=config)
    print(row["sample_non_compliant"])
    print()

In [ ]:
# we want to look at the full dataframe for the most common reasons for non-compliance
# so it can be opened in the data wrangler
# get least compliant config
least_compliant_config = results.sort_values(by="avg_compliance").iloc[0].name
least_compliant_df = dfs[least_compliant_config]
print(f"Least compliant df is for model {least_compliant_config['language_model']} on {least_compliant_config}")

In [ ]:
least_compliant_df.sample(10)

#### Sampling compliant responses

In [ ]:
# for each df, sample 10 non-compliant examples
def sample_compliant(df):
    try:
        return df[df["compliance"] == True].sample(10)["response"]
    except:
        return None

fill_df_with_function(dfs, sample_compliant, "sample_compliant", results)

In [ ]:
# show the sample non-compliant examples
for config, row in results.iterrows():
    pretty_print_config(config=config)
    print(row["sample_compliant"])
    print()

## Distribution & Behavior

In [ ]:
# what are the cheap behaviors conditional on compliance?
for config, df in dfs.items():
    pretty_print_config(config=config)
    display(pd.DataFrame(df[df['compliance'] == True][["last_word_repeated", "any_word_repeated"]].mean() * 100))

In [ ]:
# what is distribution of responses?
for config, df in dfs.items():
    pretty_print_config(config=config)
    print(df["response"].value_counts(normalize=True).head(10) * 100)
    print()